In [5]:
pip install pyportfolioopt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.1/220.1 kB 8.3 MB/s eta 0:00:00


In [6]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pypfopt.hierarchical_portfolio import HRPOpt

class AntonacciDualMomentumHRP:
    def __init__(self):
        self.assets = {
            'SP500': '^GSPC',
            'EAFE': 'EFA',
            'BONDS': 'AGG',
            'BTC': 'BTC-USD',
            'GOLD': 'EGLN.L',
            'TBILL': '^IRX'
        }

    def get_asset_data(self, start_date=None, end_date=None):
        if end_date is None:
            end_date = datetime.now()
        if start_date is None:
            start_date = end_date - timedelta(days=450)

        data = {}
        for asset_name, ticker in self.assets.items():
            try:
                asset = yf.Ticker(ticker)
                hist = asset.history(start=start_date, end=end_date)
                if not hist.empty:
                    data[asset_name] = hist['Close']
            except Exception as e:
                print(f"Error downloading {asset_name}: {str(e)}")

        return pd.DataFrame(data)

    def calculate_returns(self, data, lookback_period=252):
        returns_12m = data.pct_change(periods=lookback_period)
        latest_returns = returns_12m.iloc[-1]
        absolute_momentum = latest_returns > latest_returns['TBILL']
        relative_momentum = latest_returns.rank(ascending=False)

        return pd.DataFrame({
            'Returns_12m': latest_returns,
            'Absolute_Momentum': absolute_momentum,
            'Relative_Rank': relative_momentum
        })

    def calculate_hrp_weights(self, data, risk_assets_only=True):
        # Calculate daily returns
        returns = data.pct_change().dropna()

        if risk_assets_only:
            # Remove TBILL and BONDS for risk-on allocation
            returns = returns.drop(['TBILL', 'BONDS'], axis=1)

        # Calculate HRP weights
        hrp = HRPOpt(returns)
        weights = hrp.optimize()

        return pd.Series(weights)

    def get_portfolio_allocation(self, signals, data):
        # Check if we should be in risk-off mode
        if not any(signals['Absolute_Momentum']):
            return {'BONDS': 1.0}

        # If in risk-on mode, calculate HRP weights for risk assets
        risk_weights = self.calculate_hrp_weights(data, risk_assets_only=True)

        # Filter for assets with positive absolute momentum
        valid_assets = signals[signals['Absolute_Momentum']].index
        valid_assets = [asset for asset in valid_assets if asset not in ['TBILL', 'BONDS']]

        if not valid_assets:
            return {'BONDS': 1.0}

        # Normalize weights for valid assets only
        final_weights = {}
        total_weight = sum(risk_weights[valid_assets])

        if total_weight > 0:
            for asset in valid_assets:
                final_weights[asset] = risk_weights[asset] / total_weight
        else:
            # Fallback to equal weights if HRP fails
            weight = 1.0 / len(valid_assets)
            final_weights = {asset: weight for asset in valid_assets}

        return final_weights

    def analyze_strategy(self, months=12):
        end_date = datetime.now()
        start_date = end_date - timedelta(days=months*31)

        data = self.get_asset_data(start_date, end_date)
        signals = self.calculate_returns(data)
        allocation = self.get_portfolio_allocation(signals, data)

        report = pd.DataFrame({
            'Asset': signals.index,
            'Return_12m(%)': (signals['Returns_12m'] * 100).round(2),
            'Absolute_Momentum': signals['Absolute_Momentum'],
            'Relative_Rank': signals['Relative_Rank'],
            'Allocation(%)': [allocation.get(asset, 0) * 100 for asset in signals.index]
        })

        return {
            'signals': signals,
            'allocation': allocation,
            'report': report,
            'data': data
        }

# Example usage
model = AntonacciDualMomentumHRP()
results = model.analyze_strategy()
print("\nMomentum and Allocation Report:")
print(results['report'].to_string())

<ipython-input-6-e964b2fa21ce>:37: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns_12m = data.pct_change(periods=lookback_period)
<ipython-input-6-e964b2fa21ce>:50: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = data.pct_change().dropna()



Momentum and Allocation Report:
       Asset  Return_12m(%)  Absolute_Momentum  Relative_Rank  Allocation(%)
SP500  SP500           5.15               True            3.0      28.860458
EAFE    EAFE          -2.44               True            5.0      29.397894
BONDS  BONDS          -1.59               True            4.0       0.000000
BTC      BTC          68.10               True            1.0       2.816499
GOLD    GOLD           9.30               True            2.0      38.925150
TBILL  TBILL          -6.87              False            6.0       0.000000
